---------------
# Data acquisition
---------------------

## Data Source
-----------------------------

- **Getting CSSE data on github**

    source: https://github.com/CSSEGISandData/COVID-19 by Johns Hopkins CSSE


- **Gather state testing data thru API**

    source: https://covidtracking.com/data 
    - US Testing in Time Series https://covidtracking.com/api/us/daily.csv
    - States Historical Data https://covidtracking.com/api/states/daily.csv


- **Webscrap korean testing data**

    Source: CDC of South Korean
    - sample data https://www.cdc.go.kr/board/board.es?mid=&bid=0030&act=view&list_no=366735
    
    
- **Webscrap LA community level data**

    Source: The Department of Public Health of Los Angeles County
    - sample data http://publichealth.lacounty.gov/phcommon/public/media/mediapubhpdetail.cfm?prid=2298
    
---------------------------------

## Web scraping
-----------
**Preparation**

Firstly, create a webscrap class including functions for getting tables, get text and get urls. Such class definition is stored as `./src/web_scraping`

In [10]:
from src.web_scaping import webscrap
import pandas as pd
import csv
import datetime
from src.cleaning_helper import *

**Gather data with webscape class**

Many experts treated South Korea as a best practice in controlling COVID-19 and an good model to follow. Hence, in this study we will compare COVID-19 data between US and South Korea.

Unfortunately, there isn't an existing data source for South Korea testing data for the coronavirus. We utilized web scaping technique to collect data from press release of each day.


In [ ]:
# Gather all urls for all korean press releases

url_p1 = "https://www.cdc.go.kr/board.es?mid=&bid=0030&nPage="
url_p2 = (str(i) for i in list(range(1,25)))

url_lst = []

for page in url_p2:
    nws = webscrap(url_p1 + page)
    nws.start(random_headers = True)
    url_lst.append(nws.get_urls())

with open("url_k_t.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(url_lst2)
    

In [ ]:
# Gather Table from each page

url_p1 = "https://www.cdc.go.kr"
out_lst = []

for page in urls2:
    nws = webscrap(url_p1 + page)
    nws.start(random_headers = True)
    out_lst.append(nws.get_table())

with open("South_Korea_History data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(out_lst)
    

Not only the big picture in a country scale that we care, we also wants to provide a LA picture community level data. Similarly, we webscaped community level data from press release from Department of Public Health of Los Angeles County.

In [4]:
# collect LA data

url = "http://publichealth.lacounty.gov/phcommon/public/media/mediapubhpdetail.cfm?prid="
url_lst = []
for i in list(range(2304,2267,-1)):
    url_lst.append(url + str(i))
out = []
for url in url_lst:
    nws = webscrap(url)
    nws.start(random_headers = True)
    temp_lst = []
    text = nws.get_text(elements = ["ul", "li"])
    if text:
        temp_lst.append(url[-4:])
        temp_lst.append(nws.get_table(index=1)[0][0])
        temp_lst.append(text)
        out.append(temp_lst)

# Save as csv

with open("LA_backup.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(out)


-----------------------------
## Data Cleaning
--------------------

Create a data pipeline to process raw data, since raw data were in different structure. We will process the data in different pipelines.

### LA data

In [6]:
# data pipeline after Mar 30
out2 = []
for item in out:
    temp_lst = [item[1].split("\r\n\t")[1]]
    if len(item[2]) > 0:
        temp_lst.append(lacounty_daily_text_2_table(item[2]))
    out2.append(temp_lst)

In [ ]:
# data pipeline before mar 29
out3 = []
for item in out[15:]:
    temp_lst = [item[1].split("\r\n\t")[1]]
    if len(item[2]) > 0:
        temp_lst.append(lacounty_daily_text_2_table2(item[2]))
    out3.append(temp_lst)

In [13]:
# Since Long Beach and Pasadena have independent public health department, need to get them seperately

LB = find_LB_or_PD(out, "Long Beach")[:-6]
PD = find_LB_or_PD(out, "Pasadena")[:-6]
NLB = LB[:-5]
NLB.extend([["city2","Long Beach"]])
NLB.extend(LB[-5:])
NPD = PD[:-5]
NPD.extend([["city2","Pasadena"]])
NPD.extend(PD[-5:])

In [14]:
# merge into a pandas format
LA_data = {}
for i in out2[:15]:
    if len(i[1]):
        LA_data[i[0]] = i[1]
for i in out3:
    if len(i[1]):
        LA_data[i[0]] = i[1]

df = pd.DataFrame(LA_data['April 09, 2020'], columns= ["city", 'April 09, 2020'] )
df.set_index(["city"])

for key in LA_data.keys():
    df2 = pd.DataFrame(LA_data[key], columns= ["city", key])
    df2.set_index(["city"])
    df = pd.merge(df, df2, how='left', left_on="city", right_on="city")

In [15]:
df.head(3)

,city,"April 08, 2020_x","April 08, 2020_y","April 07, 2020","April 06, 2020","April 05, 2020","April 04, 2020","April 03, 2020","April 02, 2020","April 01, 2020","March 31, 2020","March 30, 2020","March 29, 2020","March 28, 2020","March 27, 2020","March 26, 2020","March 25, 2020","March 24, 2020","March 23, 2020","March 22, 2020"
0,City of Agoura Hills,19,19,19,18,18,17,16,14,10,11,9,9,9,6,NaN,NaN,NaN,NaN,NaN
1,City of Alhambra,26,26,24,22,21,19,17,15,13,13,11,10,10,8,NaN,NaN,NaN,NaN,NaN
2,City of Arcadia,17,17,17,16,14,13,13,12,12,10,9,9,8,6,NaN,NaN,NaN,NaN,NaN


After checking the merged dataset, some data was not properly merge since the city name format has been change since March 26, 2020. We need do minor fix on the dataset.

In [16]:
df.drop(["April 09, 2020_y", "March 26, 2020", "March 25, 2020", "March 24, 2020", "March 23, 2020", "March 22, 2020"], axis=1, inplace=True)
df.rename({"April 09, 2020_x":"April 09, 2020"}, axis=1, inplace=True )
df["city2"] = df["city"].apply(lambda x: aka_name(x))

In [17]:
# fix data for the first 5 day data
key_list = list(LA_data.keys())[-5 : ]
for key in key_list:
    df2 = pd.DataFrame(LA_data[key], columns= ["city2", key])
    df2["city2"] = df2["city2"].apply(lambda x : x.strip("*"))
    df2.set_index(["city2"])
    df = pd.merge(df, df2, how='left', left_on="city2", right_on="city2")
    
df.set_index("city", inplace=True)

In [18]:
df.head(3)

,"April 08, 2020","April 07, 2020","April 06, 2020","April 05, 2020","April 04, 2020","April 03, 2020","April 02, 2020","April 01, 2020","March 31, 2020","March 30, 2020","March 29, 2020","March 28, 2020","March 27, 2020",city2,"March 26, 2020","March 25, 2020","March 24, 2020","March 23, 2020","March 22, 2020"
city,,,,,,,,,,,,,,,,,,,
City of Agoura Hills,19,19,18,18,17,16,14,10,11,9,9,9,6,Agoura Hills,5,3,3,1,NaN
City of Alhambra,26,24,22,21,19,17,15,13,13,11,10,10,8,Alhambra,8,6,5,4,NaN
City of Arcadia,17,17,16,14,13,13,12,12,10,9,9,8,6,Arcadia,6,4,4,3,2


In [19]:
# Update value for Long Beach and Pasadena
ndf = pd.DataFrame(NLB).set_index(0).transpose()
df.loc["Unincorporated - Long Beach",:] = ndf.values.tolist()[0]
ndf2 = pd.DataFrame(NPD).set_index(0).transpose()
df.loc["City of Pasadena",:] = ndf2.values.tolist()[0]
df.tail(3)

,"April 08, 2020","April 07, 2020","April 06, 2020","April 05, 2020","April 04, 2020","April 03, 2020","April 02, 2020","April 01, 2020","March 31, 2020","March 30, 2020","March 29, 2020","March 28, 2020","March 27, 2020",city2,"March 26, 2020","March 25, 2020","March 24, 2020","March 23, 2020","March 22, 2020"
city,,,,,,,,,,,,,,,,,,,
Unincorporated - Wiseburn,--,--,--,--,--,--,--,--,--,--,--,--,--,Wiseburn,NaN,NaN,NaN,NaN,NaN
- Under Investigation,732,599,599,614,606,518,505,394,309,222,184,154,143,- Under Investigation,141,56,62,61,38
City of Pasadena,80,72,58,58,37,37,37,33,25,10,10,9,9,Pasadena,9,7,6,3,3


The LA dataset were cleaned and we can save it as a backup.

In [787]:
# making a csv backup
df.to_csv("la_community_0408.csv", index=True)

### Korea data

In [380]:
with open("./Data/Testing/South_Korea_History data.csv", "r", newline="") as f:
    reader = csv.reader(f)
    k_data = list(reader)

In [394]:
# data pipeline for Korea Data

tmp_lst = []
for i in k_data:
    if len(i)>3:
        tmp_lst.append(i[3])
# tmp_lst = list(i[3] if len(i)>3 for i in k_data)
tmp_lst2 = []
for item in tmp_lst:
    tmp_lst3 = item.replace(', ', '').replace("'", '').replace("]", '').split("\\n")[1:]
    if tmp_lst3[0] not in ["discharged", "Busan"]:
        tmp_lst2.append(tmp_lst3)

for i in range(len(tmp_lst2)):
    for j in range(len(tmp_lst2[i])):
        tmp_lst2[i][j] = (tmp_lst2[i][j].replace("As of 0:00", '').replace("As of 9:00", '')
                          .replace("As of 16:00", '').replace("As of 09:00", '')
                          .replace("As of 00:00", '').replace("As of 11:30", '')
                          .replace("As of 16:30", '').replace("As of 11:00", '')
                         .replace(".","").replace(",","").replace("\\xa0","")
                          .replace("March","Mar").replace("April","Apr")
                          .strip().strip("*"))
                          
    if len(tmp_lst2[i][0].strip()) == 0:
        tmp_lst2[i] = tmp_lst2[i][1:]
    if len(tmp_lst2[i][2].strip()) == 0:
        tmp_lst2[i].remove("")
    if  " (" in tmp_lst2[i][0]:
        tmp_lst2[i][0] = tmp_lst2[i][0][: tmp_lst2[i][0].find(" (")]

## keep only one daily data
tmp_lst4 = [tmp_lst2[0]]
for i in range(2, len(tmp_lst2)):
    if tmp_lst2[i][0] != tmp_lst2[i-1][0] and tmp_lst2[i][0] != tmp_lst2[i-2][0]:
        tmp_lst4.append(tmp_lst2[i])

df = pd.DataFrame(tmp_lst4)
df.to_csv("k_data_0406_raw.csv")        

In [91]:
# df = pd.read_csv("./Data/Testing/k_data_0406_raw.csv")

In [417]:
### fix mismatch record
df = df.set_index(0)
df2 = df.loc["13 Mar":"20 Feb", 7:8]
df2.rename(columns={7: 6, 8: 7}, inplace=True)
df.loc["13 Mar":"20 Feb", 6:7] = df2
mismatch_date_lst = ["19 Feb", "18 Feb", "16 Feb", "14 Feb", "12 Feb", "10 Feb", "8 Feb"]
for date in mismatch_date_lst:
    value_lst = df.loc[date, 2:6].tolist()
    value_lst.insert(0, int(value_lst[0]) + int(value_lst[1]))
    df.loc[date, 2:7] = value_lst
    
# impute missing Feb 15th Data with real data
df.loc["15 Feb"] = [7734, 28, 19, 9 , 0, 558, 7148,0]

df.loc["19 Feb":"8 Feb", 5] = 0
df.drop(8, axis=1, inplace=True)

# convert to readable column name
n_columns=["Total","PCR_Confirmed","PCR_Discharged","PCR_Isolated","PCR_Deceased","Being_tested","Tested_negative"]
col_dict = dict((i , n_columns[int(i)-1]) for i in df.columns)
df.rename(col_dict, axis=1, inplace=True)
df.to_csv("korea_data.csv")